In [14]:
import threading
import time

# Shared database resource with a lock
database = {"account_A": 0}
lock = threading.Lock()  # Lock to ensure Two-Phase Locking

# Update in transaction_TX function
def transaction_TX(amount, operation):
    """
    Transaction TX: Performs the user-specified operation (add or deduct) with Two-Phase Locking.
    """
    print("\n[TX] Starts")

    # Growing Phase: Acquire the lock before reading and modifying the balance
    lock.acquire()
    print("[TX] Acquired lock")

    time.sleep(1)  # TX starts at t1
    local_balance = database["account_A"]
    print(f"[TX - t1] Reads balance: {local_balance}")

    time.sleep(1)  # TX operates at t2
    if operation == "deduct":
        local_balance -= amount
        print(f"[TX - t2] Deducts {amount}, new local balance: {local_balance}")
    elif operation == "add":
        local_balance += amount
        print(f"[TX - t2] Adds {amount}, new local balance: {local_balance}")

    time.sleep(3)  # TX writes back at t6
    database["account_A"] = local_balance
    print(f"[TX - t6] Writes back balance: {database['account_A']}")

    # Shrinking Phase: Release the lock after completing the transaction
    lock.release()
    print("[TX] Released lock")

# Update in transaction_TY function
def transaction_TY(amount, operation):
    """
    Transaction TY: Performs the user-specified operation (add or deduct) with Two-Phase Locking.
    """
    time.sleep(2)  # TY starts at t3
    print("\n[TY] Starts")

    # Growing Phase: Acquire the lock before reading and modifying the balance
    lock.acquire()
    print("[TY] Acquired lock")

    local_balance = database["account_A"]
    print(f"[TY - t3] Reads balance: {local_balance}")

    time.sleep(1)  # TY operates at t4
    if operation == "deduct":
        local_balance -= amount
        print(f"[TY - t4] Deducts {amount}, new local balance: {local_balance}")
    elif operation == "add":
        local_balance += amount
        print(f"[TY - t4] Adds {amount}, new local balance: {local_balance}")

    time.sleep(2)  # TY writes back at t7
    print(f"[TY - t7] Writes back balance: {local_balance}")  # Ensures t7 prints after t6
    database["account_A"] = local_balance

    # Shrinking Phase: Release the lock after completing the transaction
    lock.release()
    print("[TY] Released lock")


# User inputs
print("=== Lost Update Problem Simulation with Two-Phase Locking ===")
initial_balance = int(input("Enter the initial balance of account A: "))

# Input for TX
print("\n[TX Configuration]")
tx_operation = input("Enter the operation for TX (add/deduct): ").strip().lower()
tx_amount = int(input(f"Enter the amount to {tx_operation} by TX: "))

# Input for TY
print("\n[TY Configuration]")
ty_operation = input("Enter the operation for TY (add/deduct): ").strip().lower()
ty_amount = int(input(f"Enter the amount to {ty_operation} by TY: "))

# Shared database resource
database = {"account_A": initial_balance}

# Threads for concurrent transactions
tx_thread = threading.Thread(target=transaction_TX, args=(tx_amount, tx_operation))
ty_thread = threading.Thread(target=transaction_TY, args=(ty_amount, ty_operation))

# Start transactions
tx_thread.start()
ty_thread.start()

# Wait for both to finish
tx_thread.join()
ty_thread.join()

# Final balance
print(f"\nFinal balance after simulation: {database['account_A']}")

# Explanation of Lost Update Problem with Two-Phase Locking
print("\n=== Explanation of Lost Update Problem with Two-Phase Locking ===")
if tx_operation == "add":
    tx_effect = f"adds {tx_amount}"
elif tx_operation == "deduct":
    tx_effect = f"deducts {tx_amount}"

if ty_operation == "add":
    ty_effect = f"adds {ty_amount}"
elif ty_operation == "deduct":
    ty_effect = f"deducts {ty_amount}"

print(
    f"The issue arises because TX {tx_effect} and TY {ty_effect} concurrently, but with Two-Phase Locking:\n"
    f"- At t1, TX acquires the lock and reads the balance of {initial_balance}.\n"
    f"- TX then modifies the balance locally at t2 and holds the lock.\n"
    f"- At t3, TY waits because the lock is held by TX.\n"
    f"- At t4, TY acquires the lock after TX releases it, then reads the balance and operates on it.\n"
    f"- At t6, TX writes the updated balance after acquiring the lock.\n"
    f"- At t7, TY writes its updated balance after acquiring the lock.\n"
    f"By using Two-Phase Locking, both transactions safely acquire and release locks in a controlled manner, ensuring that no updates are lost, unlike in the Lost Update Problem."
)


=== Lost Update Problem Simulation with Two-Phase Locking ===
Enter the initial balance of account A: 300

[TX Configuration]
Enter the operation for TX (add/deduct): add
Enter the amount to add by TX: 50

[TY Configuration]
Enter the operation for TY (add/deduct): deduct
Enter the amount to deduct by TY: 30

[TX] Starts
[TX] Acquired lock
[TX - t1] Reads balance: 300

[TY] Starts
[TX - t2] Adds 50, new local balance: 350
[TX - t6] Writes back balance: 350
[TX] Released lock
[TY] Acquired lock
[TY - t3] Reads balance: 350
[TY - t4] Deducts 30, new local balance: 320
[TY - t7] Writes back balance: 320
[TY] Released lock

Final balance after simulation: 320

=== Explanation of Lost Update Problem with Two-Phase Locking ===
The issue arises because TX adds 50 and TY deducts 30 concurrently, but with Two-Phase Locking:
- At t1, TX acquires the lock and reads the balance of 300.
- TX then modifies the balance locally at t2 and holds the lock.
- At t3, TY waits because the lock is held by TX

In [ ]:
a